M. Amintoosi

## تولید خروجی لاتک با دستورات پایتون

In [86]:
from pylatex import Document, Package, Command, NoEscape, Section, Table, Center, Tabular, Figure, Label
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io
from skimage.util import random_noise

time = np.arange(0, 10, 0.01)
frequency = 0.2
params_list = ['First Noise Level', 'Second Noise Level']
params_list_names = [s.replace(' ','-') for s in params_list]
noise_list = [0.05,0.15]

y1 = np.sin(2*np.pi*frequency*time+noise_list[0]*np.random.randn(len(time)))
y2 = np.sin(2*np.pi*frequency*time+noise_list[1]*np.random.randn(len(time)))

output_path = 'outputs'
texFileName ='test'

plt.figure()
plt.plot(time, y1)
plt.savefig(os.path.join(output_path, params_list_names[0]+'.pdf'))
plt.close()

plt.figure()
plt.plot(time, y2)
plt.savefig(os.path.join(output_path, params_list_names[1]+'.pdf'))
plt.close()

geometry_options = {"tmargin": "1cm", "lmargin": "1cm"}
doc = Document(texFileName, documentclass="article",geometry_options=geometry_options)
doc.packages.append(Package('amsmath'))
doc.packages.append(Package('float'))
doc.packages.append(Package('hyperref', 'colorlinks'))

doc.preamble.append(Command('title', 'Python 2 LaTeX'))
doc.preamble.append(Command('author', 'Mahmood Amintoosi'))
doc.preamble.append(Command('date{}'))
doc.append(NoEscape(r'\maketitle'))
## Section 1: Add noise to sin(x)
with doc.create(Section(NoEscape("Add Noise to Sin"))):

    with doc.create(Table(position='ht')) as table:
        with table.create(Center()) as centered:
            table.add_caption(NoEscape('The effect of noise levels on $\sin(x)$'))
            with centered.create(Tabular('|c|c|')) as tabular:
                tabular.add_hline()
                tabular.add_row('Noise Level', 'Figure')
                tabular.add_hline()
                for j, param in enumerate(params_list):
                    tabular.add_row(noise_list[j], NoEscape('Fig. \\ref{fig:param' + str(j)+'}'))
                    tabular.add_hline()    

    for j, param in enumerate(params_list_names):        
        with doc.create(Figure(position='H')) as plot:
            plot.add_image(filename=os.path.join(output_path, param + '.pdf'), width=NoEscape('0.4\\textwidth'))
            plot.add_caption(params_list[j])
            plot.append(Label('fig:param' + str(j)))

## Section 2: Add noise to images
noizy_images_output_paths = []
for nz in noise_list:
    noizy_images_output_path = '{}/{:03d}'.format(output_path,int(nz*100))
    noizy_images_output_paths.append(noizy_images_output_path)
    if not os.path.exists(noizy_images_output_path):
        os.makedirs(noizy_images_output_path)
n_nz = len(noise_list)
with doc.create(Section("Add noise to images")):

    with doc.create(Table(position='ht')) as table:
        with table.create(Center()) as centered:
            table.add_caption('Add noise to images')
            with centered.create(Tabular('|c|'+'c|'*n_nz)) as tabular:
                tabular.add_hline()
                header = ['Original Image'] + ['sigma = '+str(nz) for nz in noise_list]
                tabular.add_row(header)
                tabular.add_hline()

                folder = "images"
                for f_name in os.listdir(folder):
                    if f_name.endswith(".jpg"):
                        filename = folder+'/'+f_name
                        img = io.imread(filename)
                        images_list = []
                        images_list.append(NoEscape('\includegraphics[width=0.2\\textwidth]{{{}}}'.format(filename)))
                        for sigma, noise_folder in zip(noise_list,noizy_images_output_paths):
                            noisy = random_noise(img, var=sigma**2)
                            noisy_images_path = noise_folder+'/'+f_name                        
                            io.imsave(noisy_images_path,(noisy * 255).astype(np.uint8))
                            images_list.append(NoEscape('\includegraphics[width=0.2\\textwidth]{{{}}}'.format(noisy_images_path)))

                        tabular.add_row(images_list)
                        tabular.add_hline()    

doc.generate_tex()

# doc.generate_pdf(texFileName, clean_tex=False)

# For other tex engines and double process
doc.generate_pdf(texFileName, clean_tex=False, clean=False, compiler='xelatex')
doc.generate_pdf(texFileName, clean_tex=False, compiler='xelatex')